**PascalVOC to Yolo Conversion**

In [1]:
import argparse
import os
import xml.etree.ElementTree as ET

def convert_coordinates(size, box):
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def convert_annotation(input_dir, output_dir, image_name):
    in_file = open(os.path.join(input_dir, image_name[:-4] + '.xml'))
    out_file = open(os.path.join(output_dir, image_name[:-4] + '.txt'), 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult_element = obj.find('difficult')
        if difficult_element is not None:
            difficult = difficult_element.text
        else:
            difficult = '0'

        cls = obj.find('name').text
        if cls not in classes or int(difficult) == 1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert_coordinates((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

def main(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith('.xml'):
            convert_annotation(input_dir, output_dir, filename)

if __name__ == '__main__':

    input_dir = '/content/drive/MyDrive/datasets/labels/train'
    output_dir = '/content/drive/MyDrive/datasets/yolo'

    classes = ['person', 'hard-hat', 'gloves', 'mask', 'glasses', 'boots', 'vest', 'ppe-suit', 'ear-protector', 'safety-harness']

    main(input_dir, output_dir)

**For Cheking All labels are paired with Images.**

In [3]:
import os

image_dir = '/content/drive/MyDrive/datasets/images/train'
label_dir = '/content/drive/MyDrive/datasets/yolo'

image_files = set(f.replace('.jpg', '') for f in os.listdir(image_dir) if f.endswith('.jpg'))
label_files = set(f.replace('.txt', '') for f in os.listdir(label_dir) if f.endswith('.txt'))

missing_labels = image_files - label_files
missing_images = label_files - image_files

if missing_labels:
    print(f"Missing labels for images: {missing_labels}")

if missing_images:
    print(f"Missing images for labels: {missing_images}")

if not missing_labels and not missing_images:
    print("All labels and images are correctly paired.")


All labels and images are correctly paired.
